# Earth Engine Tile Layers



## Authentication

The first step is to choose a project and login to Google Cloud.

First install ee api if not already installed
`pip install eartheninge-api`


## Define service account credentials

The service account key should be saved in the `.auth` folder. For security, it is not saved on the repo. 

In [82]:
import ee
service_account = 'tnc-data-v1@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '../.auth/tncKey.json')
ee.Initialize(credentials)

## Test the API by printing the elevation of Mount Rainer

In [34]:
mt_rainer = ee.Geometry.Point([-121.756163642,46.85166326])
img = ee.Image("USGS/NED")
elevation_meters = (image.reduceRegion(ee.Reducer.first(),mt_rainer))
print(elevation_meters.getInfo())

{'elevation': 4312.91259765625}


set up url for tiles 

In [40]:
project_folder = "projects/ee-tacoma-watershed/assets/Tacoma_watershed_assets/"

lidar_dsm_image = ee.Image(project_folder+"lidar_dsm").selfMask()
visParams =  {"palette":[
  "E3F0FE",
  "1F4E00",
  "2C5100",
  "3A5600",
  "445900",
  "4F5C02",
  "5A6005",
  "65660C",
  "736D18",
  "7E7423",
  "8A7B2E",
  "94823A",
  "9F8945",
  "AC9253",
  "B79A5E",
  "C3A36A"
], 
"min": 0, "max": 600
          }

map_id_dict = lidar_dsm_image.getMapId(visParams)
tiles=map_id_dict['tile_fetcher'].url_format

print(tiles)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/maps/c5f453af81fef7d9d83cf6db395a547f-012a67d85f83f7fac2768a4d283d0aea/tiles/{z}/{x}/{y}


**Info on visualization parameters**

```params	ImageVisualizationParameters	
The visualization parameters as a (client-side) JavaScript object. For Images and ImageCollections:

  - image (JSON string) The image to render.

  - version (number) Version number of image (or latest).

  - bands (comma-separated strings) Comma-delimited list of band names to be mapped to RGB.

  - min (comma-separated numbers) Value (or one per band) to map onto 00.

  - max (comma-separated numbers) Value (or one per band) to map onto FF.

  - gain (comma-separated numbers) Gain (or one per band) to map onto 00-FF.

  - bias (comma-separated numbers) Offset (or one per band) to map onto 00-FF.

  - gamma (comma-separated numbers) Gamma correction factor (or one per band)

  - palette (comma-separated strings) List of CSS-style color strings (single-band previews only).

  - opacity (number) a number between 0 and 1 for opacity.

  - format (string) Either "jpg" or "png".```

# Test out on a plotly map

In [80]:

Tacoma_center = pd.DataFrame({"lat":47.25632945593931,"lon":-122.435107421875},index=[0])

import plotly.express as px

fig = px.scatter_mapbox(Tacoma_center, lat="lat", lon="lon", zoom=8, height=300)

              
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_layers=[
        {
            "sourcetype": "raster",
            "sourceattribution": "City of Tacoma",
            "source": [
                tiles
            ]
        }
      ])

fig.show()